In [1]:
import dask.array as da
from dask_jobqueue import SGECluster
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client, wait

In [2]:
cluster = SGECluster(
    cores=32,
    walltime="1500000",
    processes=32,
    memory="192GB",  # for memory requests, this must be specified
    interface="ib0",  # specify the network interface to use,
    job_extra_directives=["-pe mvapich2-sam 32", "-S /bin/bash"],
    job_script_prologue=[
        "export OMP_NUM_THREADS=1",
        "export MKL_NUM_THREADS=1",
        "export NUMEXPR_NUM_THREADS=1",
        "export OPENBLAS_NUM_THREADS=1",
    ],
    death_timeout=150,
    shebang="#!/bin/bash",
    local_directory="/scratch03.local/smohr/dask",
    log_directory="/scratch03.local/smohr/dask/logs",
)
client = Client(cluster)

cluster.scale(96)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SGECluster
Dashboard: http://172.9.60.181:8787/status,
Dashboard: http://172.9.60.181:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.9.60.181:35253,Workers: 0
Dashboard: http://172.9.60.181:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
import dask
import random



# Create a Dask dataframe with 10 million rows
df = dd.from_array(da.random.uniform(-1,1,10_000_000_000, chunks="1gb"), columns=["Random"])
# Get size of dataframe


#Save to orc
_ = df.to_orc(path="/scratch03.local/smohr/dask/orc",engine="pyarrow", storage_options={"compression":"ZLIB"},write_index=False)


In [8]:
print("gb", df.memory_usage().compute()["Random"] / 1e9)
print(df.sum().compute())


gb 80.0
Random   -27219.693171
dtype: float64


In [10]:


a = dd.read_orc("/scratch03.local/smohr/dask/orc", engine="pyarrow")

In [11]:
a.sum().compute()

Task exception was never retrieved
future: <Task finished name='Task-153' coro=<Client._gather.<locals>.wait() done, defined at /data.nst/smohr/anaconda3/envs/dask/lib/python3.9/site-packages/distributed/client.py:2141> exception=AllExit()>
Traceback (most recent call last):
  File "/data.nst/smohr/anaconda3/envs/dask/lib/python3.9/site-packages/distributed/client.py", line 2150, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-154' coro=<Client._gather.<locals>.wait() done, defined at /data.nst/smohr/anaconda3/envs/dask/lib/python3.9/site-packages/distributed/client.py:2141> exception=AllExit()>
Traceback (most recent call last):
  File "/data.nst/smohr/anaconda3/envs/dask/lib/python3.9/site-packages/distributed/client.py", line 2150, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-155' coro=<Client._gather.<locals>.wait() done, defined

Random   -27219.693171
dtype: float64

In [8]:
cluster.close()